In [35]:
!git clone https://github.com/stellargraph/stellargraph.git

Cloning into 'stellargraph'...
Updating files:  93% (367/394)
Updating files:  94% (371/394)
Updating files:  95% (375/394)
Updating files:  96% (379/394)
Updating files:  97% (383/394)
Updating files:  98% (387/394)
Updating files:  99% (391/394)
Updating files: 100% (394/394)
Updating files: 100% (394/394), done.


In [1]:
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import stellargraph as sg
from tensorflow import keras
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

from stellargraph import StellarGraph
from stellargraph.layer import Node2Vec, link_classification, GraphSAGE
from stellargraph.data import BiasedRandomWalk, EdgeSplitter, UniformRandomWalk, UnsupervisedSampler
from stellargraph.mapper import  Node2VecLinkGenerator, Node2VecNodeGenerator
from stellargraph.mapper import  GraphSAGENodeGenerator, GraphSAGELinkGenerator

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Задание - Предсказание уровня экспресси белка

<img src='https://www.researchgate.net/publication/313504607/figure/fig3/AS:459880453677066@1486655453033/Protein-protein-interaction-PPI-network-of-DEGs-by-STRING-The-interaction-score-was.png'>




<div class="alert alert-info">
<b>Про биологию</b>
    
Экспрессия — процесс, в ходе которого наследственная информация от гена (последовательности нуклеотидов ДНК) преобразуется в функциональный продукт — белок. Уровнем экспрессии называют - количество белка, производящегося в этом процессе. Чем выше экспрессия белка, тем большее количество этого белка появляется в клетках человека. 
    
    

<div class="alert alert-info">    
<b>Важность задачи</b>
    
Существует множество причин необходимости в знании уровня экспресии белка. Например - это позволяет ученым разрабатывать лекарственные средства и оптимизировать их разработку. Теперь вам предстоит побыть в роли биоинформатика и помочь науке!
    
</div>


<div class="alert alert-info">
<b>Про Датасет</b>
    
Датасет представляет собой граф взаимойдествия белков. Где узлы это белки, взаимодействие между белками это ребро. 

Для каждого белка известен уровень его экспрессии. Ниже приведен список ребер `edges`. Информация по экспрессии белков, разбитая на `train` и `test`.
   
    
</div>

In [2]:
#Список ребер графа 

edges = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/edges.csv", sep=",") # Подгрузим данные
edges.head()

,node_1,node_2
0,344,50
1,344,153
2,344,532
3,344,679
4,344,986


In [3]:
#Подгрузим тренирочную выборку
train = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/train.csv", sep=",") # Подгрузим данные
train.head()

,target,node
0,0.251968,11142
1,0.689541,2243
2,0.678245,15514
3,0.272500,20944
4,0.248888,8721


In [4]:
# Подгрузим отложенную выборку для валидации
test = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/test.csv", sep=",")
test.head()

,target,node
0,0.279231,817
1,0.380795,9574
2,0.686527,1607
3,0.303594,4782
4,0.367374,24125


<div class="alert alert-info">
<b>Про Задачу</b>
    
Вам предлагается предсказать экспрессию белков (`target`) по приведенным данным для отложенной выборки. Ответы в отложенной выборке `test` даны вам для самостоятельной валидации.


    
   
    

<div class="alert alert-info">
<b>Замечание и комментарии</b>
    
    

По ряду причин датасет был упрощен так, чтобы выполнялись следующие условия:
* у графа одна компонента связанности. 
* удалены слишком крупные хабы
* плотность связей графа уменьшена
* решить задачу можно классическими ML подходами
    
   

<div class="alert alert-info">
<b>Оценка результатов</b>
    


Оценка точности модели будет оцениваться по метрике MSE на отложенной выборке `test`
        
</div>

<div class="alert alert-info">
<b>Автор задачи</b>

По всем дополнительным вопросами писать Александру Миленькину
* Телеграмм: Alerin75infskin
* Почта: milenkin.aa@phystech.edu
        
</div>

## Получение базовых признаков узлов

In [18]:
G = StellarGraph(edges=edges, source_column='node_1', target_column='node_2')

In [24]:
G_nx = nx.from_pandas_edgelist(edges, source='node_1', target='node_2')

In [163]:
# посчитаем различные метрики центральности для узлов и посмотрим, какие из них дадут прирост в качестве моделей

In [35]:
# degree centrality - чем больше соседей у узла, тем больше значение
degree_centrality = nx.degree_centrality(G_nx)
degree_centrality_train = [round(degree_centrality[i], 8) for i in train['node'].tolist()]
degree_centrality_test = [round(degree_centrality[i], 8) for i in test['node'].tolist()]

In [70]:
# betweenness_centrality - чем больше проходящих путей через узел, тем больше значение
betweenness_centrality = nx.betweenness_centrality(G_nx)
betweenness_centrality_train = [round(betweenness_centrality[i], 8) for i in train['node'].tolist()]
betweenness_centrality_test = [round(betweenness_centrality[i], 8) for i in test['node'].tolist()]

In [69]:
# closeness_centrality - чем ближе узел ко всем остальным узлам, тем больше значение
closeness_centrality = nx.closeness_centrality(G_nx)
closeness_centrality_train = [round(closeness_centrality[i], 8) for i in train['node'].tolist()]
closeness_centrality_test = [round(closeness_centrality[i], 8) for i in test['node'].tolist()]

In [77]:
# была идея добавить признак принадлежности узла к тому или иному сообществу, но код не выполнился даже через 3 дня

#from networkx.algorithms import community

#communities_generator = community.girvan_newman(G_nx)

#top_level_communities = next(communities_generator)
#next_level_communities = next(communities_generator)
#communities = sorted(map(sorted, next_level_communities))

KeyboardInterrupt: 

In [65]:
df_metrics = {'method': [], 'MSE': []} # датасет для сравнения mse по каждому из методов

In [66]:
def function_add_metrics(method, mse):
    df_metrics['method'] += [method]
    df_metrics['MSE'] += [round(mse, 3)]
    
    return df_metrics

## Решение№1 - Node2Vec

In [36]:
batch_size = 64
epochs = 2
emb_size = 128
walk_number = 10
walk_length = 5

In [37]:
rw = BiasedRandomWalk(G, n=walk_number, length=walk_length, p=0.5, q=2.0) # определяем параметры случайных блужданий
unsupervised_samples = UnsupervisedSampler(G, nodes=list(G.nodes()), walker=rw) # трансформирование результатов случайных блужданий в выборки вида "id узла 1 - id узла 2 - есть связь или нет"
# ВОПРОС: могут ли в unsupervised_samples быть связи между узлами, несвязанными напрямую? То есть получается, что если два узла появляются в контексте одного блуждания, они являются связанными?

generator = Node2VecLinkGenerator(G, batch_size) # генератор для прогнозирования наличия или отсутствия связей между узлами
node2vec = Node2Vec(emb_size, generator=generator) # смысл модели node2vec - узлы, находящиеся близко в графе, должны иметь схожие эмбеддинги, и нужно подобрать такие параметры эмбеддингов, чтобы наилучшим образом решалась задача классификации связей между узлами
x_inp, x_out = node2vec.in_out_tensors() # возвращаются входные векторы двух узлов (x_inp) и их эмбеддинги (x_out)?

In [38]:
prediction

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'reshape_2')>

In [39]:
prediction = link_classification(output_dim=1, output_act='sigmoid', edge_embedding_method='dot')(x_out) # функция (слой нейронки), которая уже по эмбеддингам двух узлов в результате их аггрегации предсказывает, есть ли связь между ними (то есть prediction вида 0 или 1) 
model = keras.Model(inputs=x_inp, outputs=prediction) # модель, на вход которой подаются id двух узлов, а на выходе 0 или 1
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.binary_crossentropy) # параметры оптимизации модели

link_classification: using 'dot' method to combine node embeddings into edge embeddings


In [40]:
%%time
history = model.fit(generator.flow(unsupervised_samples), epochs=epochs, verbose=1, shuffle=True)
# модель подбирает веса для наилучшшего представления эмбеддингов двух узлов, чтобы получился искомый predicition?
# ВОПРОС: получается, что эмбеддинги в x_out - это какие-то изначальные представления узлов, а задача модели в том, чтобы их наилучшим образом перетрансформировать под задачу классификации? 

Epoch 1/2
12500/12500 [==============================] - 407s 32ms/step - loss: 0.7348
Epoch 2/2
12500/12500 [==============================] - 448s 36ms/step - loss: 0.7526
CPU times: total: 1h 14min 51s
Wall time: 17min 55s


In [41]:
x_inp[0]

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_3')>

In [42]:
# здесь вообще не очень понятно, что происходит, где здесь используются результаты предыдущей модели model?
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src) # модель, которая по id узла выдает его эмбеддинг, вопрос, почему эта модель не обучается, как она это просто выдает?

In [63]:
print(G.nodes().values)

[  344 27073 17468 ... 11542 20708  1185]


In [43]:
node_gen_train = Node2VecNodeGenerator(G, batch_size).flow([a for a in train['node'].values]) # генератор для решения задачи классификации узлов, подается список узлов
node_embeddings_train = np.row_stack([embedding_model.predict(b[0], verbose=0) for b in node_gen_train]) # модель предсказывает эмбеддинги для каждого узла
print(node_embeddings_train.shape)

(8000, 128)


In [44]:
node_gen_test = Node2VecNodeGenerator(G, batch_size).flow([a for a in test['node'].values])
node_embeddings_test = np.row_stack([embedding_model.predict(b[0], verbose=0) for b in node_gen_test])
print(node_embeddings_test.shape)

(2000, 128)


In [57]:
node_embeddings_train[1]

array([-0.37047124,  0.40225986, -0.36509988, -0.551221  , -0.33363846,
        0.03577701,  0.7108227 ,  0.37107694, -0.6011538 ,  0.19487159,
       -0.39022028, -0.6150001 , -0.21881334, -0.7961887 ,  0.42601338,
        0.11953713,  0.78667104,  0.71261764,  0.36819306,  0.8995808 ,
       -0.03337247,  0.2667061 , -0.19468747,  0.357742  , -0.07335925,
        0.94272536, -0.66263896,  0.23686177, -0.4668501 , -0.10128147,
       -0.37212694, -0.21456368,  0.9551181 ,  0.6276452 ,  1.1120037 ,
       -0.32135385, -0.63197875, -0.22613822, -0.32825536,  0.73370016,
       -0.6087741 ,  0.905746  , -0.15506123, -0.02499713, -0.5758703 ,
        0.5401822 , -0.05734826,  0.47034052,  0.566748  ,  0.588192  ,
       -0.21350044, -0.7962067 ,  0.4040464 , -0.6956961 ,  0.5128672 ,
        0.63982487, -0.79054177,  0.7968507 , -0.29668894, -0.574168  ,
        0.5183935 , -0.9071786 ,  0.6027369 , -0.55443716,  0.9523302 ,
        0.79611266, -0.88880676,  0.99493665, -0.60083437, -0.51

In [67]:
rf = RandomForestRegressor()
rf.fit(node_embeddings_train, train['target']) # обучаем любую модель регрессии по входным данным эмббединг узла - экспрессия белка
Y_pred = rf.predict(node_embeddings_test)
df_metrics = function_add_metrics('node2vec_rf', mean_squared_error(test['target'], Y_pred))
print(mean_squared_error(test['target'], Y_pred))

0.8039469666013713


In [68]:
rf = RandomForestRegressor()
node_embeddings_dc_train = [np.append(i, degree_centrality_train[k]) for k, i in enumerate(node_embeddings_train)]
node_embeddings_dc_test = [np.append(i, degree_centrality_test[k]) for k, i in enumerate(node_embeddings_test)]

rf.fit(node_embeddings_dc_train, train['target']) # добавили признак degree_centrality
Y_pred = rf.predict(node_embeddings_dc_test)
df_metrics = function_add_metrics('node2vec_degree_centrality_rf', mean_squared_error(test['target'], Y_pred))
print(mean_squared_error(test['target'], Y_pred))

0.024299923212626095


In [79]:
rf = RandomForestRegressor()
node_embeddings_bc_train = [np.append(i, betweenness_centrality_train[k]) for k, i in enumerate(node_embeddings_train)]
node_embeddings_bc_test = [np.append(i, betweenness_centrality_test[k]) for k, i in enumerate(node_embeddings_test)]

rf.fit(node_embeddings_bc_train, train['target']) # добавили признак betweenness_centrality
Y_pred = rf.predict(node_embeddings_bc_test)
df_metrics = function_add_metrics('node2vec_betweenness_centrality_rf', mean_squared_error(test['target'], Y_pred))
print(mean_squared_error(test['target'], Y_pred))

0.03644316877214989


In [80]:
rf = RandomForestRegressor()
node_embeddings_cc_train = [np.append(i, closeness_centrality_train[k]) for k, i in enumerate(node_embeddings_train)]
node_embeddings_cc_test = [np.append(i, closeness_centrality_test[k]) for k, i in enumerate(node_embeddings_test)]

rf.fit(node_embeddings_cc_train, train['target']) # добавили признак closeness_centrality
Y_pred = rf.predict(node_embeddings_cc_test)
df_metrics = function_add_metrics('node2vec_closeness_centrality_rf', mean_squared_error(test['target'], Y_pred))
print(mean_squared_error(test['target'], Y_pred))

0.06886534777743225


## Решение№2 - GraphSAGE

In [135]:
batch_size = 64
epochs = 2
walk_number = 5
walk_length = 10
num_samples = [10, 5]
layer_sizes = [50, 50]

In [83]:
G.node_features()

array([], shape=(10000, 0), dtype=float32)

In [118]:
features = [[round(degree_centrality[i], 8), round(betweenness_centrality[i], 8), round(closeness_centrality[i], 8)] for i in G.nodes()]

In [121]:
node_data.head()

,0,1,2
344,0.014801,0.000057,0.422970
27073,0.017202,0.000117,0.449333
17468,0.005000,0.000017,0.395452
12471,0.018602,0.000139,0.484190
3125,0.013501,0.000117,0.475351


In [120]:
node_data = pd.DataFrame(features, index=G.nodes().tolist())

In [122]:
G_feat = StellarGraph.from_networkx(G_nx, node_features=node_data)

In [123]:
G_feat.node_features()

array([[1.4801480e-02, 5.7490000e-05, 4.2296955e-01],
       [1.7201720e-02, 1.1685000e-04, 4.4933268e-01],
       [5.0005000e-03, 1.6570000e-05, 3.9545184e-01],
       ...,
       [5.2005202e-03, 1.5740001e-05, 4.1861340e-01],
       [5.8005801e-03, 1.8659999e-05, 4.3738243e-01],
       [5.1005101e-03, 1.0690000e-05, 4.5342827e-01]], dtype=float32)

In [136]:
unsupervised_samples = UnsupervisedSampler(G_feat, nodes=list(G_feat.nodes()), length=walk_length, number_of_walks=walk_number)
generator = GraphSAGELinkGenerator(G_feat, batch_size, num_samples)
graphsage = GraphSAGE(layer_sizes, generator=generator, bias=True) # смысл модели grapsage - подобрать такие параметры аггрегации признаков соседних узлов и эмбеддингов узлов, чтобы наилучшим образом решалась задача классификации связей между узлами
x_inp, x_out = graphsage.in_out_tensors()

C:\Users\koren\AppData\Roaming\Python\Python39\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [137]:
prediction = link_classification(output_dim=1, output_act='sigmoid', edge_embedding_method='ip')(x_out) 
model = keras.Model(inputs=x_inp, outputs=prediction)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.binary_crossentropy)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [138]:
%%time
history = model.fit(generator.flow(unsupervised_samples), epochs=epochs, verbose=1, shuffle=True)

Epoch 1/2
14063/14063 [==============================] - 1361s 97ms/step - loss: 0.7170
Epoch 2/2
14063/14063 [==============================] - 1315s 93ms/step - loss: 0.7070
CPU times: total: 49min 45s
Wall time: 45min 15s


In [146]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_24 (InputLayer)          [(None, 10, 3)]      0           []                               
                                                                                                  
 input_25 (InputLayer)          [(None, 50, 3)]      0           []                               
                                                                                                  
 input_27 (InputLayer)          [(None, 10, 3)]      0           []                               
                                                                                                  
 input_28 (InputLayer)          [(None, 50, 3)]      0           []                               
                                                                                            

In [139]:
x_inp_src = x_inp[0::2] #ВОПРОС: почему берется не x_inp[0], а x_inp[0::2]?
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [141]:
node_gen_train = GraphSAGENodeGenerator(G_feat, batch_size, num_samples).flow([a for a in train['node'].values])
node_embeddings_train = np.row_stack([embedding_model.predict(b[0], verbose=0) for b in node_gen_train])
print(node_embeddings_train.shape)

(8000, 50)


In [142]:
node_gen_test = GraphSAGENodeGenerator(G_feat, batch_size, num_samples).flow([a for a in test['node'].values])
node_embeddings_test = np.row_stack([embedding_model.predict(b[0], verbose=0) for b in node_gen_test])
print(node_embeddings_test.shape)

(2000, 50)


In [143]:
node_embeddings_train

array([[-0.03225483,  0.02726044,  0.13855854, ...,  0.05831769,
         0.0185952 ,  0.1042701 ],
       [-0.16371809,  0.04585166,  0.10159466, ..., -0.00919825,
         0.0498893 ,  0.08238328],
       [-0.4280258 ,  0.14900573,  0.1258986 , ...,  0.02780342,
        -0.12011281, -0.07667533],
       ...,
       [ 0.12050819, -0.12060038, -0.10879087, ...,  0.11193641,
        -0.00743191,  0.14440128],
       [-0.05559229, -0.0648063 , -0.10359725, ...,  0.16485873,
        -0.08172357,  0.12803346],
       [-0.07918573,  0.08120842,  0.11340284, ...,  0.0609542 ,
         0.03742252,  0.22922885]], dtype=float32)

In [144]:
rf = RandomForestRegressor()
rf.fit(node_embeddings_train, train['target']) # обучаем любую модель регрессии по входным данным эмббединг узла - экспрессия белка
Y_pred = rf.predict(node_embeddings_test)
df_metrics = function_add_metrics('graphsage_rf', mean_squared_error(test['target'], Y_pred))
print(mean_squared_error(test['target'], Y_pred))

0.01576652577507869


## Решение№3 - Word2Vec

In [149]:
rw = BiasedRandomWalk(G, n=walk_number, length=walk_length, p=0.5, q=2.0)

In [150]:
walks = rw.run(nodes=list(G.nodes()))

In [151]:
str_walks = [[str(a) for a in b] for b in walks] # считаем, что каждое блуждание - это некоторый "текст", где токены - это узлы

In [152]:
model = Word2Vec(str_walks, vector_size=50, workers=3) # каждый узел кодируется эмбеддингом размера 50

In [153]:
node_embeddings_train = np.stack([model.wv[str(a)] for a in train['node'].values]) # получаем эмбеддинги для каждого узла

In [154]:
node_embeddings_test = np.stack([model.wv[str(a)] for a in test['node'].values])

In [155]:
node_embeddings_train

array([[ 0.00877515, -0.04256485,  0.01174157, ..., -0.11223593,
         0.05709378,  0.18553546],
       [-0.00419964, -0.09361814,  0.04775286, ..., -0.25761497,
         0.13223149,  0.41730443],
       [-0.01185365, -0.07294144,  0.09714877, ..., -0.29331705,
         0.15621929,  0.45000085],
       ...,
       [-0.01458532, -0.04289096,  0.06652514, ..., -0.19667906,
         0.09996622,  0.27759495],
       [-0.03885319, -0.07478883,  0.08752935, ..., -0.2931588 ,
         0.15089116,  0.42845595],
       [-0.00140917, -0.06446022,  0.0115719 , ..., -0.1527885 ,
         0.06095127,  0.21689488]], dtype=float32)

In [156]:
rf = RandomForestRegressor()
rf.fit(node_embeddings_train, train['target']) # обучаем любую модель регрессии по входным данным эмббединг узла - экспрессия белка
Y_pred = rf.predict(node_embeddings_test)
df_metrics = function_add_metrics('word2vec_rf', mean_squared_error(test['target'], Y_pred))
print(mean_squared_error(test['target'], Y_pred))

0.16468123893106504


In [157]:
rf = RandomForestRegressor()
node_embeddings_dc_train = [np.append(i, degree_centrality_train[k]) for k, i in enumerate(node_embeddings_train)]
node_embeddings_dc_test = [np.append(i, degree_centrality_test[k]) for k, i in enumerate(node_embeddings_test)]

rf.fit(node_embeddings_dc_train, train['target']) # добавили признак degree_centrality
Y_pred = rf.predict(node_embeddings_dc_test)
df_metrics = function_add_metrics('word2vec_degree_centrality_rf', mean_squared_error(test['target'], Y_pred))
print(mean_squared_error(test['target'], Y_pred))

0.01934439670670096


In [158]:
rf = RandomForestRegressor()
node_embeddings_bc_train = [np.append(i, betweenness_centrality_train[k]) for k, i in enumerate(node_embeddings_train)]
node_embeddings_bc_test = [np.append(i, betweenness_centrality_test[k]) for k, i in enumerate(node_embeddings_test)]

rf.fit(node_embeddings_bc_train, train['target']) # добавили признак betweenness_centrality
Y_pred = rf.predict(node_embeddings_bc_test)
df_metrics = function_add_metrics('word2vec_betweenness_centrality_rf', mean_squared_error(test['target'], Y_pred))
print(mean_squared_error(test['target'], Y_pred))

0.019854026110661345


In [159]:
rf = RandomForestRegressor()
node_embeddings_cc_train = [np.append(i, closeness_centrality_train[k]) for k, i in enumerate(node_embeddings_train)]
node_embeddings_cc_test = [np.append(i, closeness_centrality_test[k]) for k, i in enumerate(node_embeddings_test)]

rf.fit(node_embeddings_cc_train, train['target']) # добавили признак closeness_centrality
Y_pred = rf.predict(node_embeddings_cc_test)
df_metrics = function_add_metrics('word2vec_closeness_centrality_rf', mean_squared_error(test['target'], Y_pred))
print(mean_squared_error(test['target'], Y_pred))

0.02993836605589177


## Результаты

In [160]:
df_metrics = pd.DataFrame(df_metrics)

In [161]:
df_metrics.sort_values(['MSE'], ascending=True).reset_index().drop('index', axis=1)

,method,MSE
0,graphsage_rf,0.016
1,word2vec_degree_centrality_rf,0.019
2,word2vec_betweenness_centrality_rf,0.020
3,node2vec_degree_centrality_rf,0.024
4,word2vec_closeness_centrality_rf,0.030
5,node2vec_betweenness_centrality_rf,0.036
6,node2vec_closeness_centrality_rf,0.069
7,word2vec_rf,0.165
8,node2vec_rf,0.804


### Можно сделать выводы, что добавление метрик центральности для узлов значительно повышает качество предсказаний по сравнению с "пустыми" методами. При этом лучшие результаты показывает простая метрика, зависящая от количества соседей узла.
### Это можно объяснить тем, что экспрессия белка может сильно зависеть от его "популярности" и степени связанности с другими белками